In [1]:
from pyspark.sql import SparkSession
!pip install ipython-autotime
%load_ext autotime

from pyspark import SparkConf, SparkContext
spark = SparkSession.builder.appName('SVM-PSO').getOrCreate()

df = spark.read.csv('Dataset/breast_cancer.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- diagnosis: string (nullable = true)
 |-- radius_mean: double (nullable = true)
 |-- texture_mean: double (nullable = true)
 |-- perimeter_mean: double (nullable = true)
 |-- area_mean: double (nullable = true)
 |-- smoothness_mean: double (nullable = true)
 |-- compactness_mean: double (nullable = true)
 |-- concavity_mean: double (nullable = true)
 |-- concave points_mean: double (nullable = true)
 |-- symmetry_mean: double (nullable = true)
 |-- fractal_dimension_mean: double (nullable = true)
 |-- radius_se: double (nullable = true)
 |-- texture_se: double (nullable = true)
 |-- perimeter_se: double (nullable = true)
 |-- area_se: double (nullable = true)
 |-- smoothness_se: double (nullable = true)
 |-- compactness_se: double (nullable = true)
 |-- concavity_se: double (nullable = true)
 |-- concave points_se: double (nullable = true)
 |-- symmetry_se: double (nullable = true)
 |-- fractal_dimension_se: double (nullable = true)
 |-- radi

In [2]:
import pandas as pd
df = df.drop('id')
X = df.drop('_c32')
f = X.drop('diagnosis')

y = df['diagnosis']
# df.drop('age').collect()
pd.DataFrame(X.take(5), columns=X.columns).transpose()

,0,1,2,3,4
diagnosis,M,M,M,M,M
radius_mean,17.99,20.57,19.69,11.42,20.29
texture_mean,10.38,17.77,21.25,20.38,14.34
perimeter_mean,122.8,132.9,130,77.58,135.1
area_mean,1001,1326,1203,386.1,1297
smoothness_mean,0.1184,0.08474,0.1096,0.1425,0.1003
compactness_mean,0.2776,0.07864,0.1599,0.2839,0.1328
concavity_mean,0.3001,0.0869,0.1974,0.2414,0.198
concave points_mean,0.1471,0.07017,0.1279,0.1052,0.1043
symmetry_mean,0.2419,0.1812,0.2069,0.2597,0.1809


time: 4.94 s


In [3]:
from pyspark.sql.functions import when, lit
X = X.withColumn('diagnosis', when(X.diagnosis == 'B', lit(0)).otherwise(1))

time: 248 ms


In [4]:

X.groupby('diagnosis').count().toPandas()

,diagnosis,count
0,1,212
1,0,357


time: 1.84 s


In [5]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = f.schema.names, outputCol = 'features')
X = vectorAssembler.transform(X)

time: 2.8 s


In [6]:
pd.DataFrame(X.take(5), columns=X.columns).transpose()

,0,1,2,3,4
diagnosis,1,1,1,1,1
radius_mean,17.99,20.57,19.69,11.42,20.29
texture_mean,10.38,17.77,21.25,20.38,14.34
perimeter_mean,122.8,132.9,130,77.58,135.1
area_mean,1001,1326,1203,386.1,1297
smoothness_mean,0.1184,0.08474,0.1096,0.1425,0.1003
compactness_mean,0.2776,0.07864,0.1599,0.2839,0.1328
concavity_mean,0.3001,0.0869,0.1974,0.2414,0.198
concave points_mean,0.1471,0.07017,0.1279,0.1052,0.1043
symmetry_mean,0.2419,0.1812,0.2069,0.2597,0.1809


time: 444 ms


In [7]:
train, test = X.randomSplit([0.7, 0.3], seed = 42)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 426
Test Dataset Count: 143
time: 857 ms


# Logistic Regression

In [8]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# assembler = VectorAssembler(inputCols=[list_of_header_names],outputCol="features")
lr = LogisticRegression(featuresCol = 'features', labelCol = 'diagnosis')
lrModel = lr.fit(train)
predictions = lrModel.transform(test)

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="diagnosis",metricName="accuracy")
print('Accuracy', evaluator.evaluate(predictions))

Accuracy 0.9300699300699301
time: 11.2 s


# Decission Tree

In [9]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol="diagnosis", featuresCol="features")
dtModel = dt.fit(train)
predictions = dtModel.transform(test)

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="diagnosis",metricName="accuracy")
print('Accuracy', evaluator.evaluate(predictions))

Accuracy 0.965034965034965
time: 2.02 s


# Naive Bayes

In [10]:

from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(labelCol="diagnosis", featuresCol="features")
nbModel = nb.fit(train)
predictions = nbModel.transform(test)

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="diagnosis",metricName="accuracy")
print('Accuracy', evaluator.evaluate(predictions))

time: 888 ms


# SVM

In [11]:

from pyspark.ml.classification import LinearSVC

svm = LinearSVC(labelCol="diagnosis", featuresCol="features")
svmModel = svm.fit(train)
predictions = svmModel.transform(test)

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="diagnosis",metricName="accuracy")
print('Accuracy', evaluator.evaluate(predictions))

Accuracy 0.958041958041958
time: 6.36 s
